In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "introduction",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import re\n",
    "from rapidfuzz import process\n",
    "from fuzzywuzzy import fuzz\n",
    "import ipywidgets as widgets\n",
    "from IPython.display import display, HTML\n",
    "\n",
    "# Sample data for testing\n",
    "data = {\n",
    "    'IdEinsender': [1, 2, 3],\n",
    "    'Anschrift1': ['Musterstrasse 1', 'Beispielweg 2', 'Testgasse 3'],\n",
    "    'Anschrift2': ['', '', ''],\n",
    "    'Anschrift3': ['', '', ''],\n",
    "    'Strasse': ['Musterstrasse', 'Beispielweg', 'Testgasse'],\n",
    "    'Plz': [12345, 23456, 34567],\n",
    "    'Ort': ['Musterstadt', 'Beispielstadt', 'Teststadt'],\n",
    "    'Telefon': ['1234567890', '0987654321', '1122334455'],\n",
    "    'eMail': ['test1@example.com', 'test2@example.com', 'test3@example.com'],\n",
    "    'EinKurz': ['Test1', 'Test2', 'Test3'],\n",
    "    'CreatedAt': ['2021-01-01', '2021-02-01', '2021-03-01']\n",
    "}\n",
    "\n",
    "df = pd.DataFrame(data)\n",
    "\n",
    "# Function to highlight search term\n",
    "def highlight_search_term(text, search_term, color):\n",
    "    regex = re.compile(re.escape(search_term), re.IGNORECASE)\n",
    "    return regex.sub(lambda match: f'<mark style=\"background-color: {color};\">{match.group(0)}</mark>', text)\n",
    "\n",
    "# Function to highlight partial match\n",
    "def highlight_partial_match(text, search_term, color):\n",
    "    best_match = process.extractOne(search_term, [text])\n",
    "    if best_match and best_match[1] > 60:\n",
    "        return highlight_search_term(text, best_match[0], color)\n",
    "    return text\n",
    "\n",
    "# Fuzzy search function\n",
    "def fuzzy_search_optimized(search_term, df, columns):\n",
    "    df['combined'] = df[columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)\n",
    "    matches = process.extract(search_term, df['combined'], limit=100)\n",
    "    matching_indices = [match[2] for match in matches]\n",
    "    result_df = df.loc[matching_indices, columnsshown]\n",
    "    \n",
    "    for column in columns:\n",
    "        result_df[column] = result_df[column].astype(str).apply(\n",
    "            lambda x: highlight_search_term(x, search_term, 'yellow') \n",
    "                      if re.search(re.escape(search_term), x, re.IGNORECASE) \n",
    "                      else x\n",
    "        )\n",
    "    \n",
    "    for column in columns:\n",
    "        result_df[column] = result_df[column].astype(str).apply(\n",
    "            lambda x: highlight_partial_match(x, search_term, 'orange') \n",
    "                      if fuzz.partial_ratio(search_term.lower(), x.lower()) > 60 \n",
    "                      else x\n",
    "        )\n",
    "    \n",
    "    display(HTML(result_df.to_html(index=False, escape=False)))\n",
    "\n",
    "# Create a text input widget\n",
    "search = widgets.Text(\n",
    "    value='',\n",
    "    placeholder='Type a search term',\n",
    "    description='Search:',\n",
    "    disabled=False,\n",
    "    layout=widgets.Layout(width='400px')\n",
    ")\n",
    "search.style.description_width = 'initial'\n",
    "\n",
    "# Columns to be shown\n",
    "columnsshown = [\n",
    "    'IdEinsender', 'Anschrift1', 'Anschrift2', 'Anschrift3', \n",
    "    'Strasse', 'Plz', 'Ort', 'Telefon', 'eMail', 'combined', 'EinKurz', 'CreatedAt'\n",
    "]\n",
    "\n",
    "# Set up the interactive display\n",
    "output = widgets.Output()\n",
    "\n",
    "def on_search_change(change):\n",
    "    with output:\n",
    "        output.clear_output()\n",
    "        fuzzy_search_optimized(change['new'], df, ['EinKurz'])\n",
    "\n",
    "# Add event listener to the text input\n",
    "search.observe(on_search_change, names='value')\n",
    "\n",
    "# Display the widgets\n",
    "display(search, output)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
